In [ ]:
%matplotlib inline

import lsqfit
from model_avg_paper import *
from model_avg_paper.test_tmin import test_vary_tmin_SE

In [ ]:
p0_test_ME = {
    'A0': 2.0,
    'E0': 0.8,
    'A1': 10.4,
    'E1': 1.16,
}
Nt = 32
noise_params = {
    'noise_amp': 0.3,
    'noise_samples': 500,
    'frac_noise': True,
    'cross_val': False,
    'cv_frac': 0.1,
}
obs_name='E0'

correlated_data = True
rho=0.6

# Set seed for consistency of outcome
#np.random.seed(10911)  # Fig 3, subfig A; Fig 4
#np.random.seed(81890)  # Fig 3, subfig B
#np.random.seed(87414)  # Fig 3, subfig C
np.random.seed(77700)  # Fig 3, subfig D


def ME_model(x,p):
    return multi_exp_model(x,p,Nexc=2)

In [ ]:
if correlated_data:
    test_data = gen_synth_data_corr(
        np.arange(0,Nt), 
        p0_test_ME, 
        ME_model,
        rho=rho,
        **noise_params)
else:
    test_data = gen_synth_data(
        np.arange(0,Nt), 
        p0_test_ME, 
        ME_model,
        **noise_params)

In [ ]:
test_res = test_vary_tmin_SE(test_data, Nt=Nt, max_tmin=26, obs_name=obs_name, IC='AIC', 
                             cross_val=noise_params['cross_val'])
print(test_res['obs_avg'])

In [ ]:
## Figure 3

import matplotlib.ticker as ticker

gs = plt.GridSpec(2, 1, height_ratios=[3,1])
gs.update(hspace=0.06)

ax1 = plt.subplot(gs[0])

plot_gvcorr([test_res['obs_avg']], x=np.array([1.5]), color='red', markersize=7, marker='s', open_symbol=True, label='Model avg.')
plot_gvcorr(test_res['obs'], x=test_res['tmin'], label='Individual fits')

ax1.plot(np.array([-1,34]), 0*np.array([0,0])+p0_test_ME[obs_name], linestyle='--', color='k', label='Model truth')
#ax1.set_xlabel('$N_p$')
ax1.set_ylabel('$E_0$')

ax1.legend(loc='center left', bbox_to_anchor=(1,0.5))
ax1.set_xlim(0.7,27.3)

plt.setp(ax1.get_xticklabels(), visible=False)

ax2 = plt.subplot(gs[1])

p_norm = test_res['probs'] / np.sum(test_res['probs'])
Q_norm = test_res['Qs'] / np.sum(test_res['Qs'])
plt.plot(test_res['tmin'], p_norm, color='orange', label='pr$(M|D)$')
plt.plot(test_res['tmin'], Q_norm, color='blue', linestyle='-.', label='Fit p-value')  # Note: fit prob != model prob! 

tick_spacing = 4
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
plt.yticks([0,np.max(p_norm)])
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:.2f}'.format(x)))
ax2.set_xlim(0.7,27.3)




# Put a legend to the right of the current axis
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax2.set_xlabel(r'$t_{\rm min}$')
ax2.set_ylabel('p')

# Uncomment to save figure to disk
#plt.savefig('plots/exp_avg_4.pdf', bbox_inches = "tight")


In [ ]:
# Scaling w/number of samples
Nsamp_array = np.array([20, 40, 80, 160, 320, 640, 2040, 4096, 4096*2, 4096*4])
Nsamp_max = Nsamp_array[-1]

noise_params['noise_samples'] = Nsamp_max
if correlated_data:
    scale_data = gen_synth_data_corr(
        np.arange(0,Nt), 
        p0_test_ME, 
        ME_model,
        rho=rho,
        **noise_params)    
else:
    scale_data = gen_synth_data(
        np.arange(0,Nt), 
        p0_test_ME, 
        ME_model,
        **noise_params)

model_avg_vs_Nsamp = []
naive_avg_vs_Nsamp = []
fixed_tmin_vs_Nsamp = []
fixed_tmin_2_vs_Nsamp = []
fw_vs_Nsamp = []

fix_tmin = 14
fix_tmin_2 = 8


for Nsamp in Nsamp_array:
    test_data_scale = cut_synth_data_Nsamp(scale_data, Nsamp)
    test_res_scale = test_vary_tmin_SE(test_data_scale, Nt=Nt, max_tmin=Nt-4, obs_name=obs_name, IC='AIC')
    test_res_scale_naive = test_vary_tmin_SE(test_data_scale, Nt=Nt, max_tmin=Nt-4, obs_name=obs_name, 
                                             IC='naive')                                             
    
    model_avg_vs_Nsamp.append(test_res_scale['obs_avg'])
    naive_avg_vs_Nsamp.append(test_res_scale_naive['obs_avg'])
    fixed_tmin_vs_Nsamp.append(test_res_scale['obs'][fix_tmin])
    fixed_tmin_2_vs_Nsamp.append(test_res_scale['obs'][fix_tmin_2])
    fw_vs_Nsamp.append(obs_avg_full_width(test_res_scale['obs'], test_res_scale['Qs'], test_res_scale['fits'], bf_i=None))






In [ ]:
## Figure 4

plot_gvcorr(model_avg_vs_Nsamp, x=np.log(Nsamp_array)+0.1, label='Model avg. (AIC)')
plot_gvcorr(fixed_tmin_vs_Nsamp, x=np.log(Nsamp_array)+0.2, color='red', marker='s', markersize=6, label=r'Fixed $t_{\rm min} = 14$')
plot_gvcorr(fw_vs_Nsamp, x=np.log(Nsamp_array)+0.3, marker='X', markersize=8, color='orange', label='Full-width systematic')
plot_gvcorr(naive_avg_vs_Nsamp, x=np.log(Nsamp_array)+0.4, color='silver', marker='v', markersize=8, label=r'Model avg. (naive)')

plt.plot(np.arange(0,10), 0*np.arange(0,10)+p0_test_ME[obs_name], linestyle='--', color='k', label='Model truth')
plt.xlabel(r'$\log(N_s)$')
plt.ylabel(r'$E_0$')
plt.xlim(2.7,7.)
plt.ylim(0.78,0.82)


# Put a legend to the right of the current axis
ax = plt.subplot(111)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Uncomment to save figure to disk
#plt.savefig('plots/exp_N_scaling.pdf', bbox_inches = "tight")
